<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Comptage des nucléotides sur une fenêtre

Dans ce complément nous allons écrire un programme python permettant de visualiser les comptages de nucléotides, sur des fenêtres glissantes et recouvrantes, comme cela a été expliqué dans la vidéo.

Commençons comme toujours par la cellule de compatibilité python2/python3

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

### 

In [ ]:
from samples import slide_1_7

*****
*****
*****


In [ ]:
def compter_c_g(adn, debut, fin):
    c = g = 0
    for i in range(debut, fin):
        if adn[i] == 'C':
            c += 1
        elif adn[i] == 'G':
            g += 1
    return c, g

def fenetre_x_y(adn, fenetre, recouvrement):
    longueur = len(adn)
    debut = 0
    X = []
    Y = []
    while debut < longueur:
        c, g = compter_c_g(adn, debut, min(longueur, debut+fenetre))
        x = debut
        # si on n'a vraiment pas de chance il n'y a ni C ni G
        # et dans ce cas là, attention à la division par 0
        if c == 0 and g == 0:
            y = 0.
        else:
            y = (g - c) / (g + c)
        X.append(x)
        Y.append(y)
        debut += (fenetre - recouvrement)
    return X, Y

In [ ]:
def dessiner(adn, fenetre, recouvrement):
    X, Y = fenetre_x_y(adn, fenetre, recouvrement)
    plt.plot(X, Y)
    plt.show()

In [ ]:
test1 = 5*'C' + 5*'G' + 5*'C' + 5*'G' + 5*'C' + 5*'G' + 5*'C' + 5*'G'
print(test1)
dessiner(test1, 10, 5)

In [ ]:
dessiner(slide_1_7, 100, 10)

In [ ]:
dessiner(slide_1_7, 20, 5)